# Feteching data

In [1]:
# set up a token
! pip install -q kaggle
from google.colab import files

files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-10-29 12:28:54          14829  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           8839  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           3992  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3294  
promptcloud/careerbuilder-job-listing-2020

In [2]:
# download dataset
! kaggle competitions download -c sentiment-analysis-on-movie-reviews
! mkdir data
! unzip train.tsv.zip -d data
! unzip test.tsv.zip -d data
! ls ./data

  0% 0.00/494k [00:00<?, ?B/s]
100% 494k/494k [00:00<00:00, 32.0MB/s]
  0% 0.00/1.28M [00:00<?, ?B/s]
100% 1.28M/1.28M [00:00<00:00, 89.2MB/s]
  0% 0.00/583k [00:00<?, ?B/s]
100% 583k/583k [00:00<00:00, 73.3MB/s]
Archive:  train.tsv.zip
  inflating: data/train.tsv          
Archive:  test.tsv.zip
  inflating: data/test.tsv           
test.tsv  train.tsv


# Loading data

In [3]:
#Reproducing same results
import torch
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [4]:
from torchtext.legacy import data
TEXT = data.Field(tokenize='spacy', tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField()
fields = [(None, None), (None, None),('text',TEXT),('label', LABEL)]

In [5]:
#loading custom dataset
training_data=data.TabularDataset(path = './data/train.tsv',format = 'tsv',fields = fields,skip_header = True)


In [6]:

#print preprocessed text
print(vars(training_data.examples[0]))

{'text': ['A', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'], 'label': '1'}


In [7]:

import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))
valid_data,test_data = valid_data.split(split_ratio=0.5, random_state = random.seed(SEED))

In [8]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10)) 
print(LABEL.vocab.freqs.most_common(5))   

#Word dictionary
print(TEXT.vocab.stoi)   

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:22<00:00, 17913.42it/s]


Size of TEXT vocabulary: 16194
Size of LABEL vocabulary: 5
[('the', 32877), (',', 29329), ('a', 23544), ('of', 23033), ('and', 22450), ('-', 16312), ('to', 15778), ('.', 12349), ("'s", 11975), ('in', 9681)]
[('2', 55741), ('3', 23046), ('1', 19133), ('4', 6435), ('0', 4887)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f36987ff710>>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, 'a': 4, 'of': 5, 'and': 6, '-': 7, 'to': 8, '.': 9, "'s": 10, 'in': 11, 'is': 12, 'that': 13, 'it': 14, 'as': 15, 'with': 16, 'for': 17, 'its': 18, 'film': 19, 'movie': 20, 'an': 21, '`': 22, 'this': 23, 'be': 24, 'but': 25, 'on': 26, 'The': 27, 'you': 28, "n't": 29, 'by': 30, "'": 31, 'more': 32, 'his': 33, 'than': 34, 'one': 35, 'about': 36, 'not': 37, 'at': 38, 'from': 39, 'or': 40, '--': 41, 'all': 42, 'have': 43, 'are': 44, 'has': 45, 'like': 46, 'out': 47, 'A': 48, 'so': 49, 'story': 50, '-RRB-': 51, 'up': 52, 'who': 53, 'most': 54, 'good': 55, 'into': 56, 

In [9]:
vars(train_data[-1])

{'label': '2', 'text': ['on', 'its', 'own', 'terms']}

In [10]:
BATCH_SIZE = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key=lambda x: len(x.text),
    device = device)


In [12]:
# Print number of batches in each split.
print('Created `train_iterator` with %d batches!'%len(train_iterator))

for batch in train_iterator:

  # Let's check batch size.
  print('Batch size: %d\n'% len(batch.text[0]))
  print('LABEL\tLENGTH\tTEXT'.ljust(10))

  # Print each example.
  i=0
  for text, label in zip(batch.text, batch.label):
    print('%s\t%d\t%s'.ljust(10) % (label, len(text), text))
    print('Batch examples lengths: %s'.ljust(20) % str([len(text) for example in text]))
  print('\n')

  # Only look at first batch. Reuse this code in training models.
  break
   

Created `train_iterator` with 5463 batches!
Batch size: 20

LABEL	LENGTH	TEXT
tensor(0, device='cuda:0')	20	tensor([ 1722,     2,    13,   486,   108, 13097,    14,  1340,    73,  1534,
           20,   302,  1062,    21,  5432,   168,    27,    12,    68, 11838],
       device='cuda:0')  
Batch examples lengths: [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
tensor(3, device='cuda:0')	20	tensor([13009,    81,  4770, 11815,  3902,    77,   126,     1,    29,  1081,
          175,  4495,     8,  2483,     1,  2232,  4686,    17,   101,   151],
       device='cuda:0')  
Batch examples lengths: [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
tensor(1, device='cuda:0')	20	tensor([13307,     7,     5,    10,    30,   421,    43,     1,   371,    10,
            1,     6,   210,  1276,     1,     1,     5,  3038,   180,    24],
       device='cuda:0')  
Batch examples lengths: [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

# Model

In [13]:

import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        #text = [sent len, batch size]
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [1]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,630,005 trainable parameters


In [16]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.6946, -1.3275, -1.1976,  ...,  0.6238, -0.8446,  0.0270],
        [-1.1092, -0.0392, -0.5137,  ..., -0.0642,  0.9299,  0.3551],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3962, -0.0070,  0.4369,  ..., -0.3295,  0.1764,  0.0092],
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310],
        [ 0.5732, -1.0756, -0.1600,  ...,  0.4548,  0.2344,  0.0364]])

In [17]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [18]:
import torch.optim as optim
! pip install adabelief_pytorch
from adabelief_pytorch import AdaBelief

#optimizer = optim.Adam(model.parameters())

optimizer = AdaBelief(model.parameters(), lr=1e-3, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False) 

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [19]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [ ]:
all = len(test_data)
good = 0
for example in test_data:
  test_x = " ".join(vars(example)['text'])
  test_y = int(vars(example)['label'])
  pred_class = predict_class(model, test_x)
  if(int(LABEL.vocab.itos[pred_class]) == int(test_y)): good += 1 

print(f"{good}/{all}={good/all*100}")  



In [ ]:
example = input()
pred_class = predict_class(model, example)
print(f'Predicted class is: {LABEL.vocab.itos[pred_class]}')